<a href="https://colab.research.google.com/github/FullmetalDataScientist/AAIC-MCI-RD-User-Internet-Usage-Forecasting-Challenge/blob/main/AAIC2021_MCI_RD_UIUF_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('forecast', repos='http://cran.rstudio.com/')
install.packages('h2o', repos='http://cran.rstudio.com/')
install.packages('mlr', repos='http://cran.rstudio.com/')
install.packages('data.table', repos='http://cran.rstudio.com/')
install.packages('prophet', repos='http://cran.rstudio.com/')
install.packages('modeltime', repos='http://cran.rstudio.com/')
install.packages('parsnip', repos='http://cran.rstudio.com/')
install.packages('tidymodels', repos='http://cran.rstudio.com/')
install.packages('earth', repos='http://cran.rstudio.com/')
install.packages('keras', repos='http://cran.rstudio.com/')
install.packages('tidyverse', repos='http://cran.rstudio.com/')
install.packages('glue', repos='http://cran.rstudio.com/')
install.packages('forcats', repos='http://cran.rstudio.com/')
install.packages('timetk', repos='http://cran.rstudio.com/')
install.packages('tidyquant', repos='http://cran.rstudio.com/')
install.packages('tibbletime', repos='http://cran.rstudio.com/')
install.packages('recipes', repos='http://cran.rstudio.com/')

In [ ]:
############# Functions
lags <- function(x){
  lagB <- c(NA, x)
  lagA <- c(x, NA)
  DF <- as.data.frame(cbind(lagB, lagA))
  colnames(DF) <- c(paste0('n-', 1), 'n')
  return(DF)
}

smape <- function(actual, pred){
  s <- 0
  l <- length(pred)
  for (i in 1:l) {
    s <- s + abs(pred[i] - actual[i])/(abs(actual[i]) + abs(pred[i]))
  }
  return(s/l)
}

rmse <- function(actual, pred){
  s <- 0
  l <- length(pred)
  for (i in 1:l) {
    s <- s + (pred[i] - actual[i])^2
  }
  return(sqrt(s/l))
}

r2 <- function(actual, pred){
  s1 <- 0
  s2 <- 0
  l <- length(pred)
  for (i in 1:l) {
    s1 <- s1 + (actual[i] - pred[i])^2
  }
  m <- mean(actual)
  for (j in 1:l) {
    s2 <- s2 + (actual[j] - m)^2
  }
  return(1 - s1/s2)
}

In [ ]:
############# Model 1: Overall Mean per User

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

lt <- length(test$subscriber_ecid)
ld <- length(data$subscriber_ecid)
m <- aggregate(data_usage_volume ~ subscriber_ecid, data = data, FUN = "mean")
id <- m$subscriber_ecid
OM <- rep(0, lt)
test <- cbind(test, OM)

for (j in 1:lt) {
  for (i in id) {
    if (test$subscriber_ecid[j] == i) test$OM[j] <- m[m$subscriber_ecid==i, 2]
  }
}

write.csv(test, 'Model1(Week1).csv', row.names = FALSE)

In [ ]:
############# Model 2: Mean per Day of the Week

data <- read.csv('Train_Weeks_Added.csv')
test <- read.csv('Test1_Weeks_Added.csv')

m <- aggregate(data_usage_volume ~ subscriber_ecid + Week_Days, data = data, FUN = "mean")
id <- m$subscriber_ecid
day <- c("D01", "D02", "D03", "D04", "D05", "D06", "D07")
lt <- length(test$subscriber_ecid)
ld <- length(data$subscriber_ecid)
DbDM <- rep(0, lt)
test <- cbind(test, DbDM)

for (j in 1:lt) {
  for (i in id) {
    for (d in day) {
      if (test$subscriber_ecid[j] == i & test$Week_Days[j] == d) test$DbDM[j] <- m[(m$subscriber_ecid==i & m$Week_Days==d), 3]
    }
  }
}

write.csv(test, 'Model2(Week1).csv', row.names = FALSE)

In [ ]:
############# Model 3 to 11: SES, Holt-Winter's, ARIMA, TBATS, ETS, Theta, NNET, BAGGED & SplineF
require(forecast)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

lt <- length(test$subscriber_ecid)
TBATS <- rep(0, lt)
ARIMA <- rep(0, lt)
Holts <- rep(0, lt)
SES <- rep(0, lt)
ETS <- rep(0, lt)
THETA <- rep(0, lt)
NNET <- rep(0, lt)
BAGGED <- rep(0, lt)
SPLINEF <- rep(0, lt)
test <- cbind(test, TBATS, ARIMA, Holts, SES, ETS, THETA, NNET, BAGGED, SPLINEF)
id <- unique(test$subscriber_ecid)

for (i in id) {
  data_ts <- ts(data$data_usage_volume[data$subscriber_ecid == i])
  l <- length(test$subscriber_age[test$subscriber_ecid == i])
  TBATS_model <- tbats(data_ts)
  t <- forecast(TBATS_model, h = l)
  test$TBATS[test$subscriber_ecid == i] <- as.numeric(t$mean)
  ARIMA_model <- auto.arima(data_ts)
  a <- forecast(ARIMA_model, h = l)
  test$ARIMA[test$subscriber_ecid == i] <- as.numeric(a$mean)
  Holts_model <- holt(data_ts)
  h <- forecast(Holts_model, h = l)
  test$Holts[test$subscriber_ecid == i] <- as.numeric(h$mean)
  SES_model <- ses(data_ts)
  s <- forecast(SES_model, h = l)
  test$SES[test$subscriber_ecid == i] <- as.numeric(s$mean)
  ETS_model <- ets(data_ts)
  s <- forecast(ETS_model, h = l)
  test$ETS[test$subscriber_ecid == i] <- as.numeric(s$mean)
  THETA_model <- thetaf(data_ts)
  s <- forecast(THETA_model, h = l)
  test$THETA[test$subscriber_ecid == i] <- as.numeric(s$mean)
  NNET_model <- nnetar(data_ts)
  s <- forecast(NNET_model, h = l)
  test$NNET[test$subscriber_ecid == i] <- as.numeric(s$mean)
  BAGGED_model <- baggedModel(data_ts)
  s <- forecast(BAGGED_model, h = l)
  test$BAGGED[test$subscriber_ecid == i] <- as.numeric(s$mean)
  SPLINE_model <- splinef(data_ts)
  s <- forecast(SPLINE_model, h = l)
  test$SPLINEF[test$subscriber_ecid == i] <- as.numeric(s$mean)
}

write.csv(test, 'Forecast(Week1).csv', row.names = FALSE)

In [ ]:
############# Model12: GBM
library(data.table)
library(mlr)
require(h2o)

localH2o <- h2o.init(nthreads = -1)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

id <- unique(test$subscriber_ecid)
lt <- length(test$subscriber_ecid)
ld <- length(data$subscriber_ecid)

for (j in 1:ld){
  set.seed(j)
  data$data_usage_volume[j] <- data$data_usage_volume[j] + runif(1,0.0001,0.001)
}

GBM <- rep(0, lt)
test <- cbind(test, GBM)

for (i in id){
  train <- data$data_usage_volume[data$subscriber_ecid == i]
  l <- length(train)
  lu <- length(test$day[test$subscriber_ecid == i])
  train <- lags(train)
  pred <- train[(l+1), ]
  train <- train[-c(1, l+1), ]
  forecastuser <- rep(0, lu)
  setDT(train)
  setDT(pred)
  trainh2o <- as.h2o(train)
  predh2o <- as.h2o(pred)

  y <- "n"
  x <- setdiff(colnames(trainh2o), y)

  GBM_Model <- h2o.gbm(x = x,
                       y = y,
                       training_frame = trainh2o,
                       seed = 1,
                       ntrees = 1000,
                       learn_rate = 0.03,
                       stopping_rounds = 5,
                       stopping_tolerance = 1e-4,
                       stopping_metric = "RMSE",
                       score_tree_interval = 10,
                       max_depth = 13)

  for (d in 1:lu){
    forch2o <- h2o.predict(GBM_Model, predh2o)
    forecastuser[d] <- as.numeric(forch2o[1,1])
    predh2o[1,1] <- as.numeric(forch2o[1,1])
  }
  test$GBM[test$subscriber_ecid==i] <- as.vector(forecastuser)
}

write.csv(test, 'Model14(GBM).csv', row.names = FALSE)

In [ ]:
############# Model 13: XGBoost
library(data.table)
library(mlr)
require(h2o)

localH2o <- h2o.init(nthreads = -1)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

id <- unique(test$subscriber_ecid)
lt <- length(test$subscriber_ecid)
ld <- length(data$subscriber_ecid)

for (j in 1:ld){
  set.seed(j)
  data$data_usage_volume[j] <- data$data_usage_volume[j] + runif(1,0.0001,0.001)
}

XGB <- rep(0, lt)
test <- cbind(test, XGB)

for (i in id){
  train <- data$data_usage_volume[data$subscriber_ecid == i]
  l <- length(train)
  lu <- length(test$day[test$subscriber_ecid == i])
  train <- lags(train)
  pred <- train[(l+1), ]
  train <- train[-c(1, l+1), ]
  forecastuser <- rep(0, lu)
  setDT(train)
  setDT(pred)
  trainh2o <- as.h2o(train)
  predh2o <- as.h2o(pred)

  y <- "n"
  x <- setdiff(colnames(trainh2o), y)

  XGB_Model <- h2o.xgboost(x = x,
                           y = y,
                           training_frame = trainh2o,
                           seed = 1,
                           ntrees = 10000,
                           learn_rate = 0.01,
                           stopping_rounds = 5,
                           stopping_tolerance = 1e-4,
                           stopping_metric = "RMSE",
                           score_tree_interval = 10)

  for (d in 1:lu){
    forch2o <- h2o.predict(XGB_Model, predh2o)
    forecastuser[d] <- as.numeric(forch2o[1,1])
    predh2o[1,1] <- as.numeric(forch2o[1,1])
  }
  test$XGB[test$subscriber_ecid==i] <- as.vector(forecastuser)
}

write.csv(test, 'Model15(XGBoost).csv', row.names = FALSE)

In [ ]:
############# Model 14: Random Forest
library(data.table)
library(mlr)
require(h2o)

localH2o <- h2o.init(nthreads = -1)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

id <- unique(test$subscriber_ecid)
lt <- length(test$subscriber_ecid)
ld <- length(data$subscriber_ecid)

for (j in 1:ld){
  set.seed(j)
  data$data_usage_volume[j] <- data$data_usage_volume[j] + runif(1,0.0001,0.001)
}

RF <- rep(0, lt)
test <- cbind(test, RF)

for (i in id){
  train <- data$data_usage_volume[data$subscriber_ecid == i]
  l <- length(train)
  lu <- length(test$day[test$subscriber_ecid == i])
  train <- lags(train)
  pred <- train[(l+1), ]
  train <- train[-c(1, l+1), ]
  forecastuser <- rep(0, lu)
  setDT(train)
  setDT(pred)
  trainh2o <- as.h2o(train)
  predh2o <- as.h2o(pred)

  y <- "n"
  x <- setdiff(colnames(trainh2o), y)

  RF_Model <- h2o.randomForest(x = x,
                               y = y,
                               training_frame = trainh2o,
                               mtries = 1,
                               model_id = "rf_model",
                               ntrees = 200,
                               seed = 1)

  for (d in 1:lu){
    forch2o <- h2o.predict(RF_Model, predh2o)
    forecastuser[d] <- as.numeric(forch2o[1,1])
    predh2o[1,1] <- as.numeric(forch2o[1,1])
  }
  test$RF[test$subscriber_ecid==i] <- as.vector(forecastuser)
}

write.csv(test, 'Model16(RF).csv', row.names = FALSE)

In [ ]:
############# Model 15: Neural Nets
library(data.table)
library(mlr)
require(h2o)

localH2o <- h2o.init(nthreads = -1)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

id <- unique(test$subscriber_ecid)
lt <- length(test$subscriber_ecid)
ld <- length(data$subscriber_ecid)

for (j in 1:ld){
  set.seed(j)
  data$data_usage_volume[j] <- data$data_usage_volume[j] + runif(1,0.0001,0.001)
}

DNN <- rep(0, lt)
test <- cbind(test, DNN)

for (i in id){
  train <- data$data_usage_volume[data$subscriber_ecid == i]
  l <- length(train)
  lu <- length(test$day[test$subscriber_ecid == i])
  train <- lags(train)
  pred <- train[(l+1), ]
  train <- train[-c(1, l+1), ]
  forecastuser <- rep(0, lu)
  setDT(train)
  setDT(pred)
  trainh2o <- as.h2o(train)
  predh2o <- as.h2o(pred)

  y <- "n"
  x <- setdiff(colnames(trainh2o), y)

  DNN_Model <- h2o.deeplearning(x = x,
                                y = y,
                                training_frame = trainh2o,
                                reproducible = TRUE,
                                standardize = T,
                                activation = "MaxoutWithDropout",
                                epochs = 100,
                                seed = 1,
                                hidden = 13)

  for (d in 1:lu){
    forch2o <- h2o.predict(DNN_Model, predh2o)
    forecastuser[d] <- as.numeric(forch2o[1,1])
    predh2o[1,1] <- as.numeric(forch2o[1,1])
  }
  test$DNN[test$subscriber_ecid==i] <- as.vector(forecastuser)
}

write.csv(test, 'MaxWD13(Week1).csv', row.names = FALSE)

In [ ]:
############# Model 16: Prophet
library(prophet)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

id <- unique(test$subscriber_ecid)
lt <- length(test$subscriber_ecid)
ld <- length(data$subscriber_ecid)
Proph <- rep(0, lt)
test <- cbind(test, Proph)

for (i in id){
  y <- ts(data$data_usage_volume[data$subscriber_ecid == i])
  l <- length(y)
  ds <- seq(from = as.Date("1989-07-04"), length.out = l, by = 'day')
  prophet_data <- cbind(ds, y)
  for(j in 1:l){
    prophet_data[j, 1] <- as.character(ds[j])
  }
  prophet_data <- as.data.frame(prophet_data)
  prophet_model <- prophet(prophet_data, growth = 'flat')
  pred <- test$day[test$subscriber_ecid == i]
  lp <- length(pred)
  future <- make_future_dataframe(prophet_model, periods = lp, freq = "day", include_history = FALSE)
  forecast <- predict(prophet_model, future)
  Proph <- as.vector(forecast['yhat'])
  test$Proph[test$subscriber_ecid==i] <- Proph[,1]
}

write.csv(test, 'Model18(Prophet_Flat).csv', row.names = FALSE)

In [ ]:
############# Model 17: ARIMABoost
library(xgboost)
library(tidymodels)
library(modeltime)
library(tidyverse)
library(lubridate)
library(timetk)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

id <- unique(test$subscriber_ecid)
lt <- length(test$subscriber_ecid)
ARBoost <- rep(0, lt)
test <- cbind(test, ARBoost)

for(i in id){
  y <- ts(data$data_usage_volume[data$subscriber_ecid == i])
  l <- length(y)
  ds <- seq(from = as.Date("1989-07-04"), length.out = l, by = 'day')
  ARB_data <- cbind(ds, y)
  ARB_data <- as_tibble(ARB_data)
  ARB_data$ds <- as.Date(ds)
  ARB_Model <- arima_boost(min_n = 2, learn_rate = 0.03) %>%
    set_engine(engine = "auto_arima_xgboost") %>%
    fit(y ~ ds + as.numeric(ds) + factor(month(ds, label = TRUE), ordered = F), data = ARB_data)
  models_tbl <- modeltime_table(ARB_Model)
  y <- ts(test$data_usage_volume[test$subscriber_ecid == i])
  lu <- length(y)
  ds <- seq(from = as.Date(ARB_data$ds[l]), length.out = lu, by = 'day')
  ARB_data_t <- cbind(ds, y)
  ARB_data_t <- as_tibble(ARB_data_t)
  ARB_data_t$ds <- as.Date(ds)
  calibration_tbl <- models_tbl %>%
    modeltime_calibrate(new_data = ARB_data_t)
  pred <- modeltime_forecast(calibration_tbl, new_data = ARB_data_t, actual_data = ARB_data)
  test$ARBoost[test$subscriber_ecid==i] <- as.vector(pred$.value[(l+1):(l+lu)])
}

write.csv(test, 'Model19(ARIMABoost).csv', row.names = FALSE)

In [ ]:
############# Model 18: LR
library(xgboost)
library(tidymodels)
library(modeltime)
library(tidyverse)
library(lubridate)
library(timetk)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

id <- unique(test$subscriber_ecid)
lt <- length(test$subscriber_ecid)

LR <- rep(0, lt)
test <- cbind(test, LR)

for(i in id){
  y <- ts(data$data_usage_volume[data$subscriber_ecid == i])
  l <- length(y)
  ds <- seq(from = as.Date("1989-07-04"), length.out = l, by = 'day')
  LR_data <- cbind(ds, y)
  LR_data <- as_tibble(LR_data)
  LR_data$ds <- as.Date(ds)
  LR_Model <- linear_reg() %>%
    set_engine("lm") %>%
    fit(y ~ as.numeric(ds) + factor(month(ds, label = TRUE), ordered = FALSE), data = LR_data)
  models_tbl <- modeltime_table(LR_Model)
  y <- ts(test$data_usage_volume[test$subscriber_ecid == i])
  lu <- length(y)
  ds <- seq(from = as.Date(LR_data$ds[l]), length.out = lu, by = 'day')
  LR_data_t <- cbind(ds, y)
  LR_data_t <- as_tibble(LR_data_t)
  LR_data_t$ds <- as.Date(ds)
  calibration_tbl <- models_tbl %>%
    modeltime_calibrate(new_data = LR_data_t, quiet = FALSE)
  pred <- modeltime_forecast(calibration_tbl, new_data = LR_data_t, actual_data = LR_data)
  test$LR[test$subscriber_ecid==i] <- as.vector(pred$.value[(l+1):(l+lu)])
}

write.csv(test, 'Model20(LR).csv', row.names = FALSE)

In [ ]:
############# Model 19: MARS
library(xgboost)
library(tidymodels)
library(modeltime)
library(tidyverse)
library(lubridate)
library(timetk)
library(earth)

data <- read.csv('MCIRD_aaic2021_train.csv')
test <- read.csv('MCIRD_aaic2021_test_week1.csv')

id <- unique(test$subscriber_ecid)
lt <- length(test$subscriber_ecid)

MARS <- rep(0, lt)
test <- cbind(test, MARS)

for(i in id){
  y <- ts(data$data_usage_volume[data$subscriber_ecid == i])
  l <- length(y)
  ds <- seq(from = as.Date("1989-07-04"), length.out = l, by = 'day')
  MARS_data <- cbind(ds, y)
  MARS_data <- as_tibble(MARS_data)
  MARS_data$ds <- as.Date(ds)
  MARS_Model <- mars(mode = "regression") %>%
    set_engine("earth") 
  recipe_spec <- recipe(y ~ ds, data = MARS_data) %>%
    step_date(ds, features = "month", ordinal = FALSE) %>%
    step_mutate(date_num = as.numeric(ds)) %>%
    step_normalize(date_num) %>%
    step_rm(ds)
  wflw_fit_mars <- workflow() %>%
    add_recipe(recipe_spec) %>%
    add_model(MARS_Model) %>%
    fit(MARS_data)
  models_tbl <- modeltime_table(wflw_fit_mars)
  y <- ts(test$data_usage_volume[test$subscriber_ecid == i])
  lu <- length(y)
  ds <- seq(from = as.Date(MARS_data$ds[l]), length.out = lu, by = 'day')
  MARS_data_t <- cbind(ds, y)
  MARS_data_t <- as_tibble(MARS_data_t)
  MARS_data_t$ds <- as.Date(ds)
  calibration_tbl <- models_tbl %>%
    modeltime_calibrate(new_data = MARS_data_t)
  pred <- modeltime_forecast(calibration_tbl, new_data = MARS_data_t, actual_data = MARS_data)
  test$MARS[test$subscriber_ecid==i] <- as.vector(pred$.value[(l+1):(l+lu)])
}

write.csv(test, 'Model21(MARS).csv', row.names = FALSE)